V3 based on https://gemini.google.com/app/5a54aa16b879b0d3

## Creating the model
"This vignette is an example of modelling a decision tree using the rdecision package. It is based on the example given by Briggs (Box 2.3) which itself is based on a decision tree which compared oral Sumatriptan versus oral caffeine/Ergotamine for migraine. In this vignette, we consider the problem from the perspective of a provincial health department."

<figure>
<img src="img_Briggs2006_Box2-3.png" width="700" alt="Evans 1997 Figure 1"/>
<figcaption>BOX 2.3. Decision tree from Briggs et al., 2006.</figcaption>
</figure>

### Figure 1 from Evans, 1997
Below is Figure 1 from Evans, 1997 article showing the decision tree structure. 

<figure>
<img src="img_evans1997_fig01.drawio.png" width="100%" alt="Evans 1997 Figure 1"/>
<figcaption>FIGURE 1: From Evans, 1997, Figure 1.</figcaption>
</figure>

Figure 2 is the influence diagram of Figure 1. 

<figure>
<img src="img_evans1997_Influence_Diagram_V2.drawio.png" width="700" alt="Evans 1997 Figure 1"/>
<figcaption>FIGURE 2. Influence diagram derived from Evans, 1997, Figure 1.</figcaption>
</figure>

### Model variables
The following code defines the variables for cost, utility and effect that will be used in the model. There are 14 variables in total; 4 costs, 4 utilities and 6 probabilities.

For clarity, I coded some of the variables differently than in the original vignette. For example,

`p_norecurrence_relief_sumatriptan = 0.594` $= P(\text{no recurrence} \mid \text{relief}, \text{sumatriptan})$.

For example, 

`p_endures_norelief = 0.92` $= P(\text{endures} \mid \text{no relief})$ applies to both sumatriptan and caffeine/ergotamine treatment arms, so this variable appears in both arms.

## Set nodes, incoming edges, and states

In [1]:
using JuMP, HiGHS
using DecisionProgramming

In [2]:
# 2. Initialize the Influence Diagram
diagram = InfluenceDiagram()

# 3. Define Nodes
# Decision: Treatment Options
add_node!(diagram, DecisionNode("D", [], ["D1", "D2"]))

# Chance: Relief (R depends on D)
add_node!(diagram, ChanceNode("R", ["D"], ["R1", "R2"]))

# Chance: Recurrence (N depends on D and R)
# We include "NA" for the case where Relief = No;  P(N = "NA" | R = "R2") = 1
add_node!(diagram, ChanceNode("N", ["D", "R"], ["N1", "N2", "NA"]))

# Chance: No Relief Outcome (E depends on R)
# E1 = endures attack, E2 = seeks help at ED; P(E = "NA" | R = "R1") = 1
# We include "NA" for the case where R = R1 (yes relief)
add_node!(diagram, ChanceNode("E", ["R"], ["E1", "E2", "NA"]))

# Chance: ED Outcome (H depends on E)
# H1 = relief, H2 = hospitalization
# We include "NA" for the case where E = E1 (endures attack); P(H = "NA" | E = "E1") = 1
add_node!(diagram, ChanceNode("H", ["E"], ["H1", "H2", "NA"]))

# 4. Utility and Cost Nodes (defined later)
add_node!(diagram, ValueNode("U", ["D", "N", "E", "H"]))
add_node!(diagram, ValueNode("C1", ["D"]))
add_node!(diagram, ValueNode("C2", ["D", "N"]))
add_node!(diagram, ValueNode("C3", ["E"]))
add_node!(diagram, ValueNode("C4", ["H"]))

generate_arcs!(diagram)

OrderedCollections.OrderedDict{String, Utilities}()

### options for viewing diagram attributes  
`diagram.OPTION` where `OPTION` = `Nodes`, `Names`, `I_j`, `States`, `S`, `C`, `D`, or `V`.

In [3]:
diagram.I_j

OrderedCollections.OrderedDict{String, Vector{String}} with 10 entries:
  "D"  => []
  "R"  => ["D"]
  "N"  => ["D", "R"]
  "E"  => ["R"]
  "H"  => ["E"]
  "U"  => ["D", "N", "E", "H"]
  "C1" => ["D"]
  "C2" => ["D", "N"]
  "C3" => ["E"]
  "C4" => ["H"]

## Assign probabilities to variables

In [4]:
# model variables for cost
c_sumatriptan = 16.10 #
c_caffeine = 1.32 #
c_ed = 63.16 #
c_hospital = 1093.0 #

# model variables for utility
u_N1_D1 = 1.0      # A  
u_N2_D1 = 0.9      # B
u_E1_D1 = - 0.3    # C 
u_H1_E2_D1 = 0.1   # D
u_H2_E2_D1 = - 0.3 # E
u_N1_D2 = 1.0      # F  
u_N2_D2 = 0.9      # G
u_E1_D2 = - 0.3    # H
u_H1_E2_D2 = 0.1   # I
u_H2_E2_D2 = - 0.3 # J

# model variables for effect
# Sumatriptan probabilities
p_R1_D1 = 0.558
p_R2_D1 = 1 - p_R1_D1
p_N1_D1 = 0.594 
p_N2_D1 = 1 - p_N1_D1 

# Caffeine/Ergotamine probabilities
p_R1_D2 = 0.379
p_R2_D2 = 1 - p_R1_D2
p_N1_D2 = 0.703 
p_N2_D2 = 1 - p_N1_D2

# both treatment arms  
p_E1_R2 = 0.92 
p_E2_R2 = 1 - p_E1_R2
p_H1_E2 = 0.998 
p_H2_E2 = 1 - p_H1_E2

p_norecurrence_relief_caffeine = 0.703 # e11; # e12 use NA_real_ to indicate complement

p_relief_ed = .998 # e15, e17; # e16, e18 use NA_real_ to indicate complement

0.998

### Create probability matrices and assign probabilities

In [5]:
#add_node!(diagram, DecisionNode("D", [], ["S", "C"])) # Decision: Sumatriptan (S) vs Coffeine/Ergot (C)
# not applicable 

In [6]:
#add_node!(diagram, ChanceNode("R", ["D"], ["R1", "R0"])) # R1 = relief, R0 = no relief  
X_R = ProbabilityMatrix(diagram, "R")

2×2 ProbabilityMatrix{2}:
 0.0  0.0
 0.0  0.0

In [7]:
X_R = ProbabilityMatrix(diagram, "R")
X_R["D1", "R1"] = p_R1_D1
X_R["D1", "R2"] = 1 - p_R1_D1
X_R["D2", "R1"] = p_R1_D2
X_R["D2", "R2"] = 1 - p_R1_D2
add_probabilities!(diagram, "R", X_R)

2×2 Probabilities{2}:
 0.558  0.442
 0.379  0.621

In [8]:
#add_node!(diagram, ChanceNode("N", ["D", "R"], ["N1", "N0", "NA"])) # N1 = non-recurrence, N0 = recurrence, NA = no action
X_N = ProbabilityMatrix(diagram, "N")

2×2×3 ProbabilityMatrix{3}:
[:, :, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 2] =
 0.0  0.0
 0.0  0.0

[:, :, 3] =
 0.0  0.0
 0.0  0.0

In [ ]:
 X_N["D1",  "R1",  "N1"] = p_N1_D1
 X_N["D1",  "R1",  "N2"] = 1 - p_N1_D1
 X_N["D1",  "R1",  "NA"] = 0 # P(N = "NA" | R = "R1") = 0
 X_N["D1",  "R2",  "N1"] = 0 # P(N = "N1" | R = "R2") = 0
 X_N["D1",  "R2",  "N2"] = 0 # P(N = "N2" | R = "R2") = 0
 X_N["D1",  "R2",  "NA"] = 1 # P(N = "NA" | R = "R2") = 1
 X_N["D2",  "R1",  "N1"] = p_N1_D2
 X_N["D2",  "R1",  "N2"] = 1 - p_N1_D2
 X_N["D2",  "R1",  "NA"] = 0 # P(N = "NA" | R = "R1") = 0
 X_N["D2",  "R2",  "N1"] = 0 # P(N = "N1" | R = "R2") = 0
 X_N["D2",  "R2",  "N2"] = 0 # P(N = "N2" | R = "R2") = 0
 X_N["D2",  "R2",  "NA"] = 1 # P(N = "NA" | R = "R2") = 1
add_probabilities!(diagram, "N", X_N)

2×2×3 Probabilities{3}:
[:, :, 1] =
 0.594  0.0
 0.703  0.0

[:, :, 2] =
 0.406  0.0
 0.297  0.0

[:, :, 3] =
 0.0  1.0
 0.0  1.0

In [10]:
# Chance: No Relief Outcome (depends on R)
# E1 = endures attack, E2 = seeks help at ED; P(E = "NA" | R = R1) = 1
# We include "NA" for the case where R = R1 (yes relief)
X_E = ProbabilityMatrix(diagram, "E")

2×3 ProbabilityMatrix{2}:
 0.0  0.0  0.0
 0.0  0.0  0.0

In [11]:
X_E["R1", "E1"] = 0 # P(E = E1 | R = R1) = 0
X_E["R1", "E2"] = 0
X_E["R1", "NA"] = 1 # P(E = "NA" | R = R1) = 1
X_E["R2", "E1"] = p_E1_R2
X_E["R2", "E2"] = 1 - p_E1_R2
X_E["R2", "NA"] = 0 # P(E = "NA" | R = R2) = 0
add_probabilities!(diagram, "E", X_E)

2×3 Probabilities{2}:
 0.0   0.0   1.0
 0.92  0.08  0.0

In [12]:
# Chance: ED Outcome (H depends on E)
# H1 = relief, H2 = hospitalization
# We include "NA" for the case where E = E1 (endures attack); P(H = "NA" | E = "E1") = 1
X_H = ProbabilityMatrix(diagram, "H")

3×3 ProbabilityMatrix{2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [13]:
X_H["E1", "H1"] = 0 # P(H = "H1" | E = "E1") = 0
X_H["E1", "H2"] = 0 # P(H = "H2" | E = "E1") = 0
X_H["E1", "NA"] = 1 # P(H = "NA" | E = "E1") = 1
X_H["E2", "H1"] = p_H1_E2
X_H["E2", "H2"] = 1- p_H1_E2
X_H["E2", "NA"] = 0 # P(H = "NA" | E = "E2") = 0
X_H["NA", "H1"] = 0
X_H["NA", "H2"] = 0
X_H["NA", "NA"] = 1 # P(H = "NA" | E = "NA") = 1 # need to verify that this is correctly specified
add_probabilities!(diagram, "H", X_H)

3×3 Probabilities{2}:
 0.0    0.0    1.0
 0.998  0.002  0.0
 0.0    0.0    1.0

In [14]:
# add_node!(diagram, ValueNode("U", ["D", "N", "E", "H"]))
Y_U = UtilityMatrix(diagram, "U")
Y_U .= -1e9

2×3×3×3 UtilityMatrix{4}:
[:, :, 1, 1] =
 -1.0f9  -1.0f9  -1.0f9
 -1.0f9  -1.0f9  -1.0f9

[:, :, 2, 1] =
 -1.0f9  -1.0f9  -1.0f9
 -1.0f9  -1.0f9  -1.0f9

[:, :, 3, 1] =
 -1.0f9  -1.0f9  -1.0f9
 -1.0f9  -1.0f9  -1.0f9

[:, :, 1, 2] =
 -1.0f9  -1.0f9  -1.0f9
 -1.0f9  -1.0f9  -1.0f9

[:, :, 2, 2] =
 -1.0f9  -1.0f9  -1.0f9
 -1.0f9  -1.0f9  -1.0f9

[:, :, 3, 2] =
 -1.0f9  -1.0f9  -1.0f9
 -1.0f9  -1.0f9  -1.0f9

[:, :, 1, 3] =
 -1.0f9  -1.0f9  -1.0f9
 -1.0f9  -1.0f9  -1.0f9

[:, :, 2, 3] =
 -1.0f9  -1.0f9  -1.0f9
 -1.0f9  -1.0f9  -1.0f9

[:, :, 3, 3] =
 -1.0f9  -1.0f9  -1.0f9
 -1.0f9  -1.0f9  -1.0f9

In [15]:
# 2×2×3×3×3  
# ["D", "N", "E", "H"] 
Y_U["D1", "N1", "NA", "NA"] = u_N1_D1    # A
Y_U["D1", "N2", "NA", "NA"] = u_N2_D1    # B
Y_U["D1", "NA", "E1", "NA"] = u_E1_D1    # C
Y_U["D1", "NA", "E2", "H1"] = u_H1_E2_D1 # D
Y_U["D1", "NA", "E2", "H2"] = u_H2_E2_D1 # E
Y_U["D2", "N1", "NA", "NA"] = u_N1_D2    # F
Y_U["D2", "N2", "NA", "NA"] = u_N2_D2    # G
Y_U["D2", "NA", "E1", "NA"] = u_E1_D2    # H
Y_U["D2", "NA", "E2", "H1"] = u_H1_E2_D2 # I
Y_U["D2", "NA", "E2", "H2"] = u_H2_E2_D2 # J

-0.3

In [16]:
add_utilities!(diagram, "U", Y_U)

2×3×3×3 Utilities{4}:
[:, :, 1, 1] =
 -1.0f9  -1.0f9  -1.0f9
 -1.0f9  -1.0f9  -1.0f9

[:, :, 2, 1] =
 -1.0f9  -1.0f9  0.1
 -1.0f9  -1.0f9  0.1

[:, :, 3, 1] =
 -1.0f9  -1.0f9  -1.0f9
 -1.0f9  -1.0f9  -1.0f9

[:, :, 1, 2] =
 -1.0f9  -1.0f9  -1.0f9
 -1.0f9  -1.0f9  -1.0f9

[:, :, 2, 2] =
 -1.0f9  -1.0f9  -0.3
 -1.0f9  -1.0f9  -0.3

[:, :, 3, 2] =
 -1.0f9  -1.0f9  -1.0f9
 -1.0f9  -1.0f9  -1.0f9

[:, :, 1, 3] =
 -1.0f9  -1.0f9  -0.3
 -1.0f9  -1.0f9  -0.3

[:, :, 2, 3] =
 -1.0f9  -1.0f9  -1.0f9
 -1.0f9  -1.0f9  -1.0f9

[:, :, 3, 3] =
 1.0  0.9  -1.0f9
 1.0  0.9  -1.0f9

In [17]:
# add_node!(diagram, ValueNode("C1", ["D"]))
# add_node!(diagram, ValueNode("C2", ["D", "N"]))
# add_node!(diagram, ValueNode("C3", ["E"]))
# add_node!(diagram, ValueNode("C4", ["H"]))
Y_C1 = UtilityMatrix(diagram, "C1")
Y_C2 = UtilityMatrix(diagram, "C2")
Y_C3 = UtilityMatrix(diagram, "C3")
Y_C4 = UtilityMatrix(diagram, "C4")

Y_C1["D1"] = c_sumatriptan
Y_C1["D2"] = c_caffeine

Y_C2["D1", "N1"] = 0
Y_C2["D1", "N2"] = c_sumatriptan # relieved with 2nd dose of sumatriptan
Y_C2["D1", "NA"] = 0
Y_C2["D2", "N1"] = 0
Y_C2["D2", "N2"] = c_caffeine # relieved with 2nd dose of caffeine/ergotamine
Y_C2["D2", "NA"] = 0

Y_C3["E1"] = 0
Y_C3["E2"] = c_ed
Y_C3["NA"] = 0  

Y_C4["H1"] = 0
Y_C4["H2"] = c_hospital
Y_C4["NA"] = 0

add_utilities!(diagram, "C1", Y_C1)
add_utilities!(diagram, "C2", Y_C2)
add_utilities!(diagram, "C3", Y_C3)
add_utilities!(diagram, "C4", Y_C4)

3-element Utilities{1}:
    0.0
 1093.0
    0.0

In [18]:
model, z, variables = generate_model(diagram, model_type="RJT")

(A JuMP Model
├ solver: none
├ objective_sense: MAX_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 147
├ num_constraints: 334
│ ├ AffExpr in MOI.EqualTo{Float64}: 185
│ ├ AffExpr in MOI.LessThan{Float64}: 2
│ ├ VariableRef in MOI.GreaterThan{Float64}: 145
│ └ VariableRef in MOI.ZeroOne: 2
└ Names registered in the model
  └ :EV, OrderedCollections.OrderedDict{String, DecisionProgramming.DecisionVariable}("D" => DecisionProgramming.DecisionVariable("D", String[], VariableRef[D_1, D_2])), RJTVariables(Dict{String, DecisionProgramming.μVariable}("N" => DecisionProgramming.μVariable("N", VariableRef[N[D1, R1, N1] N[D1, R2, N1]; N[D2, R1, N1] N[D2, R2, N1];;; N[D1, R1, N2] N[D1, R2, N2]; N[D2, R1, N2] N[D2, R2, N2];;; N[D1, R1, NA] N[D1, R2, NA]; N[D2, R1, NA] N[D2, R2, NA]]), "D" => DecisionProgramming.μVariable("D", VariableRef[D[D1], D[D2]]), "R" => DecisionProgramming.μVariable("R", VariableRef[R[D1, R1] R[D1, R2]; R[D2, R1] R[D2, R2]]), "E" => DecisionProgramming.μVariable

In [19]:
optimizer = optimizer_with_attributes(
    () -> HiGHS.Optimizer()
)
set_optimizer(model, optimizer)
optimize!(model)

Running HiGHS 1.12.0 (git hash: 755a8e027): Copyright (c) 2025 HiGHS under MIT licence terms
MIP has 187 rows; 147 cols; 577 nonzeros; 2 integer variables (2 binary)
Coefficient ranges:
  Matrix  [2e-03, 1e+00]
  Cost    [1e-01, 1e+09]
  Bound   [1e+00, 1e+00]
  RHS     [1e+00, 1e+00]
Presolving model
26 rows, 22 cols, 64 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve reductions: rows 0(-187); columns 0(-147); nonzeros 0(-577) - Reduced to empty
Presolve: Optimal

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic;
     I => Shifting; J => Feasibility jump; L => Sub-MIP; P => Empty MIP; R => Randomized rounding;
     S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution; Y => HiGHS solution;
     Z => ZI Round; l => Trivial lower; p => Trivial point; u => Trivial upper; z => Trivial zero

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Lea

In [20]:
Z = DecisionStrategy(diagram, z)
S_probabilities = StateProbabilities(diagram, Z)
U_distribution = UtilityDistribution(diagram, Z)

UtilityDistribution([15.800000190734863, 17.100000381469727, 33.099998474121094, 79.36000061035156, 1171.9599609375], [0.40663999999999995, 0.331452, 0.22654800000000003, 0.03528927999999997, 7.072000000000001e-5])

In [21]:
print_decision_strategy(diagram, Z, S_probabilities)

┌───────────────┐
│ Decision in D │
├───────────────┤
│ D1            │
└───────────────┘


In [22]:
print_utility_distribution(U_distribution)

┌─────────────┬─────────────┐
│     Utility │ Probability │
│     Float64 │     Float64 │
├─────────────┼─────────────┤
│   15.800000 │    0.406640 │
│   17.100000 │    0.331452 │
│   33.099998 │    0.226548 │
│   79.360001 │    0.035289 │
│ 1171.959961 │    0.000071 │
└─────────────┴─────────────┘


In [23]:
print_statistics(U_distribution)
┌──────────┬────────────┐
│     Name │ Statistics │
│   String │    Float64 │
├──────────┼────────────┤
│     Mean │  31.000000 │
│      Std │   8.000000 │
│ Skewness │  -1.500000 │
│ Kurtosis │   0.250000 │
└──────────┴────────────┘

┌──────────┬─────────────┐
│     Name │  Statistics │
│   String │     Float64 │
├──────────┼─────────────┤
│     Mean │   22.474918 │
│      Std │   16.152561 │
│ Skewness │   27.052514 │
│ Kurtosis │ 1816.289127 │
└──────────┴─────────────┘


UndefVarError: UndefVarError: `┌──────────┬────────────┐` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [30]:
print_statistics(U_distribution)

┌──────────┬─────────────┐
│     Name │  Statistics │
│   String │     Float64 │
├──────────┼─────────────┤
│     Mean │   22.474918 │
│      Std │   16.152561 │
│ Skewness │   27.052514 │
│ Kurtosis │ 1816.289127 │
└──────────┴─────────────┘


## Appendix

In [ ]:
using DataFrames
d = ["D1", "D2"]
r = ["R1", "R2"]
n = ["N1", "N2", "NA"]
df = allcombinations(DataFrame, x=d, y=r, z=n)
sort!(df, [1 ,2, 3])
mat = Matrix(df);

In [ ]:
using DataFrames
d = ["D1", "D2"]
n = ["N1", "N2", "NA"]
e = ["E1", "E2", "NA"]
h = ["H1", "H2", "NA"]
df = allcombinations(DataFrame, d=d, n=n, e=e, h=h)
sort!(df, [1 ,2, 3, 4])
mat = Matrix(df);